## CODING ASSISTANT USING LANGCHAIN AND OLLAMA

In [12]:
#installing libraries
%pip install -qU langchain
%pip install -qU langchain_ollama
%pip install -qU langchain_core
%pip install -qU html2text




Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
#importing libraries
import requests
import html2text

from langchain_ollama import ChatOllama
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, ToolMessage, SystemMessage

from langchain.globals import set_verbose, set_debug
set_verbose(True)
set_debug(True)


#### Constants

In [34]:
GUIDELINES_URLS = {
    'React':'https://raw.githubusercontent.com/pillarstudio/standards/refs/heads/master/reactjs-guidelines.md',
    'Python':'https://google.github.io/styleguide/pyguide.html',
    'Java':'https://google.github.io/styleguide/javaguide.html',
    'Node':'https://raw.githubusercontent.com/Wolox/tech-guides/refs/heads/master/nodejs/docs/node-standard-and-best-practices.md',
    'Django':'https://raw.githubusercontent.com/HackSoftware/Django-Styleguide/refs/heads/master/README.md',
}

### Tool Definitions

In [39]:
@tool
def get_architectural_guidelines(component_name: str, language: str) -> str:
    """Returns architectural guidelines for a specific programming language project.

    Args:
        component_name: The name of the component.
        language: The programming language to retrieve guidelines for (e.g., 'React', 'Python', 'Java').
    """
    
    # Get the guideline URL based on the language from the dictionary
    location = GUIDELINES_URLS.get(language, "Unknown")

    # If no URL is found, return a generic best practices message
    if location == "Unknown":
        return f"No specific guidelines found for {language}. Using generic best practices for {component_name}."

    print(f"Tool call: Getting {language} architectural guidelines for {component_name} from {location}")

    # Fetch and return the architectural guidelines from the URL
    try:
        response = requests.get(location)
        response.raise_for_status()  # Check if the request was successful
        html_content = response.text
        return html2text.html2text(html_content)
    except requests.exceptions.RequestException as e:
        # Fallback if URL fetching fails
        return f"Failed to retrieve guidelines from {location}. Using generic best practices for {component_name}. Error: {str(e)}"


### Model Definition

In [30]:
llm = ChatOllama(
    model='llama3.1',
    temperature=0,
)

### Binding tools to model

In [36]:
tools = [get_architectural_guidelines]
llm_with_tools = llm.bind_tools(tools)

### System + Human Message
(User Input goes here!)

In [44]:
# System message to set the AI's role
system_message = "You are a software developer specialized in multiple programming languages, including React, Python, and Java."
messages = [SystemMessage(system_message)]

# Human message for multiple languages (User input)
human_message = "Create a sign-in component for an online banking app in react. Use best practices and excessive commenting"
messages.append(HumanMessage(human_message))

### Call the model first time with Human and System messages

In [45]:
ai_msg = llm_with_tools.invoke(messages)

[llm/start] [llm:ChatOllama] Entering LLM run with input:
{
  "prompts": [
    "System: You are a software developer specialized in multiple programming languages, including React, Python, and Java.\nHuman: Create a sign-in component for an online banking app in react. Use best practices and excessive commenting"
  ]
}
[llm/end] [llm:ChatOllama] [31.84s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "",
        "generation_info": {
          "model": "llama3.1",
          "created_at": "2024-09-30T22:28:12.054115Z",
          "message": {
            "role": "assistant",
            "content": "",
            "tool_calls": [
              {
                "function": {
                  "name": "get_architectural_guidelines",
                  "arguments": {
                    "component_name": "sign-in component",
                    "language": "React",
                    "location": "online banking app"
                  }
                }
     

### Finding the right guide based on language in prompt

In [46]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"get_architectural_guidelines": get_architectural_guidelines}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke({
        "component_name": tool_call["args"]["component_name"],
        "language": tool_call["args"]["language"],
    })
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

[tool/start] [tool:get_architectural_guidelines] Entering Tool run with input:
"{'component_name': 'sign-in component', 'language': 'React'}"
Tool call: Getting React architectural guidelines for sign-in component from https://raw.githubusercontent.com/pillarstudio/standards/refs/heads/master/reactjs-guidelines.md
[tool/end] [tool:get_architectural_guidelines] s] Exiting Tool run with output:
"# Coding Guidelines - ReactJS ## Table of Contents 1\. [Basic Rules](#basic-
rules) 1\. [Naming](#naming) 1\. [Declaration](#declaration) 1\.
[Alignment](#alignment) 1\. [Quotes](#quotes) 1\. [Spacing](#spacing) 1\.
[Props](#props) 1\. [Parentheses](#parentheses) 1\. [Tags](#tags) 1\.
[Methods](#methods) 1\. [Ordering](#ordering) ## Basic Rules \- Only include
one React component per file. \- Always use JSX syntax. \- Do not use
`React.createElement` unless you're initializing the app from a file that is
not JSX. ## Naming \- File- and component name need to be identical. \- Use
PascalCase naming

### Call model second time with System + Human + Tool messages

In [47]:
ai_msg = llm_with_tools.invoke(messages)



[llm/start] [llm:ChatOllama] Entering LLM run with input:
{
  "prompts": [
    "System: You are a software developer specialized in multiple programming languages, including React, Python, and Java.\nHuman: Create a sign-in component for an online banking app in react. Use best practices and excessive commenting\nTool: # Coding Guidelines - ReactJS ## Table of Contents 1\\. [Basic Rules](#basic-\nrules) 1\\. [Naming](#naming) 1\\. [Declaration](#declaration) 1\\.\n[Alignment](#alignment) 1\\. [Quotes](#quotes) 1\\. [Spacing](#spacing) 1\\.\n[Props](#props) 1\\. [Parentheses](#parentheses) 1\\. [Tags](#tags) 1\\.\n[Methods](#methods) 1\\. [Ordering](#ordering) ## Basic Rules \\- Only include\none React component per file. \\- Always use JSX syntax. \\- Do not use\n`React.createElement` unless you're initializing the app from a file that is\nnot JSX. ## Naming \\- File- and component name need to be identical. \\- Use\nPascalCase naming convention for filename as well as component name, 

### Output final message

In [48]:
print(ai_msg.content)

Based on the provided guidelines, here's an example of a sign-in component for an online banking app in React:

```javascript
// Signin.js
import React, { Component } from 'react';
import PropTypes from 'prop-types';

class Signin extends Component {
  constructor(props) {
    super(props);
    this.state = {
      username: '',
      password: ''
    };
  }

  _handleClick = (e) => {
    e.preventDefault();
    // Call API to authenticate user
    const { onLogin } = this.props;
    onLogin(this.state.username, this.state.password);
  };

  render() {
    return (
      <div className="signin-container">
        <h2>Sign In</h2>
        <form onSubmit={this._handleClick}>
          <input
            type="text"
            placeholder="Username"
            value={this.state.username}
            onChange={(e) => this.setState({ username: e.target.value })}
          />
          <input
            type="password"
            placeholder="Password"
            value={this.state.passw